In [1]:
import pandas as pd
from keras.models import load_model
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
import math
from sklearn.metrics import mean_squared_error
from keras.utils import plot_model
import os

In [ ]:
dpath = '/Users/hadongheon/Desktop/SKALA 1기/데이터분석 및 MLOps/ozo/server/model/stock_lstm_model_nogpu.keras'
DATA_PATH = "/Users/hadongheon/Desktop/SKALA 1기/데이터분석 및 MLOps/ozo/server/IBM_2006_to_2023_extended.csv"
PREDICTION_PLOT_PATH = "/Users/hadongheon/Desktop/SKALA 1기/데이터분석 및 MLOps/ozo/server/view-model-architecture/stock_test.png"

In [ ]:
# 모델 로딩
model = load_model(dpath)

# 데이터 로딩
dataset = pd.read_csv(DATA_PATH, index_col='Date', parse_dates=['Date'], encoding='utf-8')

# 모델 아키텍처 이미지 생성
#plot_model(model, to_file=os.path.join(IMAGE_DIR, "model.png"))
#plot_model(model, to_file=MODEL_SHAPES_PLOT_PATH, show_shapes=True)

# RMSE 계산 함수
def return_rmse(test, predicted):
    rmse = math.sqrt(mean_squared_error(test, predicted))
    result_msg = f"The root mean squared error is {rmse}."
    print(result_msg)
    return result_msg

# 예측 결과 그래프 저장 함수
def plot_predictions(test, predicted):
    plt.clf()  # 이전 그래프 초기화
    plt.plot(test, color='red', label='Real IBM Stock Price')
    plt.plot(predicted, color='blue', label='Predicted IBM Stock Price')
    plt.title('IBM Stock Price Prediction')
    plt.xlabel('Time')
    plt.ylabel('IBM Stock Price')
    plt.legend()
    plt.savefig(PREDICTION_PLOT_PATH)
    return PREDICTION_PLOT_PATH

# 데이터 전처리 및 모델 예측 실행 함수
def process(dataset):
    model = load_model(dpath)

    # 'High' 열 선택
    training_set = dataset.loc[:'2022', ["High"]].values
    test_set = dataset.loc['2023':, ["High"]].values

    # 데이터 스케일링
    sc = MinMaxScaler(feature_range=(0, 1))
    training_set_scaled = sc.fit_transform(training_set)

    # 테스트 데이터 준비
    dataset_total = pd.concat([dataset.loc[:'2022', "High"], dataset.loc['2023':, "High"]], axis=0)
    inputs = dataset_total[len(dataset_total) - len(test_set) - 60:].values
    inputs = inputs.reshape(-1, 1)
    inputs = sc.transform(inputs)

    X_test = []
    for i in range(60, len(inputs)):
        X_test.append(inputs[i-60:i, 0])
    X_test = np.array(X_test)
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # 모델 예측
    predicted_stock_price = model.predict(X_test)
    predicted_stock_price = sc.inverse_transform(predicted_stock_price)

    # 결과 시각화 및 평가
    result_visualizing = plot_predictions(test_set, predicted_stock_price)
    result_evaluating = return_rmse(test_set, predicted_stock_price)

    return result_visualizing, result_evaluating